## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key = os.getenv("GROQ_API_KEY")

'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'

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000150167B4A60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000150167B5720>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Lokesh and I am a Data Scientist")])

AIMessage(content="Hello Lokesh,\n\nIt's nice to meet you!  As a large language model, I don't have a name or personal experience like being a Data Scientist, but I can be helpful in many ways related to that field. \n\nWhat can I do for you today? Perhaps you have a data science problem you'd like to discuss, need help with code, or want to explore a particular concept?  \n\nLet me know how I can assist you!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 22, 'total_tokens': 122, 'completion_time': 0.181818182, 'prompt_time': 0.000144359, 'queue_time': 0.01429246, 'total_time': 0.181962541}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-556ce945-a4df-45ca-87b3-85a234a6484c-0', usage_metadata={'input_tokens': 22, 'output_tokens': 100, 'total_tokens': 122})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Lokesh and I am a Data Scientist"),
        AIMessage(content="Hello Lokesh,\n\nIt's nice to meet you!  As a large language model, I don't have a name or personal experience like being a Data Scientist, but I can be helpful in many ways related to that field. \n\nWhat can I do for you today? Perhaps you have a data science problem you'd like to discuss, need help with code, or want to explore a particular concept?  \n\nLet me know how I can assist you!\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Lokesh, and you are a Data Scientist!  \n\nIs there anything else you'd like to know or discuss about your work? I'm ready when you are. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 142, 'total_tokens': 186, 'completion_time': 0.08, 'prompt_time': 0.00527455, 'queue_time': 0.00915384, 'total_time': 0.08527455}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c6998123-c350-4c00-afa5-35a827426fd0-0', usage_metadata={'input_tokens': 142, 'output_tokens': 44, 'total_tokens': 186})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "chat1"}}

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Lokesh and I am a Data Scientist")],
    config=config
)

In [9]:
response.content

"Hi Lokesh, it's great to meet you! \n\nData science is a really exciting field. What kind of projects are you working on these days?  \n\nDo you have a particular area of focus within data science, like:\n\n* **Predictive modeling?**\n* **Machine learning?**\n* **Data visualization?**\n* **Something else entirely?**\n\n\nI'd love to hear more about your work and the challenges you're tackling.  Perhaps you could even share an interesting project you've been involved in!\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Lokesh. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 323, 'total_tokens': 345, 'completion_time': 0.04, 'prompt_time': 0.010128622, 'queue_time': 0.013539045, 'total_time': 0.050128622}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-53d6d98b-ae09-47ac-93c6-ddaad0a4d65a-0', usage_metadata={'input_tokens': 323, 'output_tokens': 22, 'total_tokens': 345})

In [ ]:
## change the config-->session id
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it!\n"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 😊 \n\nIs there anything I can help you with today?\n"

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nI remember that you told me earlier.  😄\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Lokesh")]})

AIMessage(content="Hi Lokesh, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 32, 'total_tokens': 68, 'completion_time': 0.065454545, 'prompt_time': 0.000328939, 'queue_time': 0.014444549, 'total_time': 0.065783484}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-57868b11-6c0f-4241-bba4-b2b3775bb258-0', usage_metadata={'input_tokens': 32, 'output_tokens': 36, 'total_tokens': 68})

In [17]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is Lokesh")],
    config=config
)

response

AIMessage(content="Hello Lokesh, it's nice to meet you! \n\nI'm happy to help answer any questions you have to the best of my ability.  \n\nWhat can I do for you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 32, 'total_tokens': 80, 'completion_time': 0.087272727, 'prompt_time': 0.000499368, 'queue_time': 0.013888441, 'total_time': 0.087772095}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8fde701b-dca9-44c0-bd91-d0f154b45a86-0', usage_metadata={'input_tokens': 32, 'output_tokens': 48, 'total_tokens': 80})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Lokesh.  I remembered from our introduction!  😄  \n\nIs there anything else I can help you with?\n'

In [20]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke({"messages":[HumanMessage(content="Hi My name is Lokesh")], "language":"Hindi"})
response.content

'नमस्ते लोकेश! \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। \n\nक्या मैं आपके लिए कुछ कर सकता हूँ? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages" # as we have multiple parameters
)

In [26]:
config = {"configurable": {"session_id": "chat4"}}
repsonse = with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Lokesh")], "language":"Telugu"},
    config=config
)
repsonse.content

'నమస్తే, లోకేష్! 😊 \n\nనాకు మీకు సహాయం చేయడం నచ్చుతుంది. ఏమి సమాధానం కావాలి? \n'

In [29]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Telugu"},
    config=config,
)

In [30]:
response.content

'మీ పేరు లోకేష్ అని తెలుసుకున్నాను! \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [31]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Lokesh"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\Study\Projects\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Study\Projects\Langchain\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mloke\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this art

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [32]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What ice cream do i like")],
        "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences.  \n\nWhat's your favorite flavor?  🍦😋\n"

In [33]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "Whats 2 + 2" 😊  \n\n\n\nLet me know if you\'d like to try another one!\n'

In [ ]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable":{"session_id":"chat5"}}

In [35]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name. 😊\n\nWould you like to tell me your name?\n"

In [36]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\nWhat can I solve for you?  \n\n"